In [3]:
import os
import h5py
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow import keras
import gc
from sys import getsizeof
import datetime
import random
import math
from sklearn.metrics import classification_report, confusion_matrix
import progressbar

In [ ]:
def load_data(res, source_labels, dataset='train', size=1., verbosity = 0, shuffle = True):
    import h5py
    import numpy 
    import gc

    data = numpy.zeros((0, res, res, res), dtype=int)
    labels = numpy.empty(0, int)
    bar = progressbar.ProgressBar(max_value=len(source_labels)-1)
  
    for i, label in enumerate(source_labels):
        filename = './output/' + label + '/output-' + dataset + '-' + str(res) + '.h5'
        file = h5py.File(filename, 'r')
        tensor_len = len(file['tensor'])
        picked_ids = range(math.ceil(tensor_len * size) - 1)
        if verbosity == 2:
            print(filename)
            print(picked_ids)
            print('Picking ' + str(math.ceil(tensor_len * size)) + ' from ' + str(len(file['tensor'])) )
        else:
            if verbosity == 1:
                bar.update(i)
        data = numpy.concatenate((data, [file['tensor'][index] for index in picked_ids]))
        labels = numpy.concatenate((labels, numpy.full(len(picked_ids), i, dtype=int)))
        file.close()
        gc.collect()
    if shuffle:
        if verbosity == 2:
            print("Shuffling data")
        rng_state = numpy.random.get_state()
        numpy.random.shuffle(data)
        numpy.random.set_state(rng_state)
        numpy.random.shuffle(labels)
    return (data, labels)



In [ ]:
def train(model, training, training_labels, res, num_classes, verbosity = 0):
    log_dir = "logs/fit/" + str(res) + '-' + str(num_classes) + '-' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    if verbosity == 1:
        print("Log file " + log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    validation_split = 0.1
    epochs = 10 * num_classes
    
    model.fit(
        prepare_data(training, num_classes), 
        training_labels, 
        epochs=epochs,
        verbose=verbosity,
        callbacks=[tensorboard_callback],
        validation_split=validation_split
    )

In [ ]:
def draw(img, expected = '', predicted = ''):
    fig = plt.figure(figsize=[10,10])
    ax = fig.gca(projection='3d')
    cube1 = (img[:,:,:] >= 1)
    ax.voxels(cube1, facecolors="blue")
    plt.title('Expected: {}\n Predicted: {}'.format(expected, predicted), y=-0.01)
    plt.show()


In [ ]:
all_labels = ['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 
                 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar','keyboard', 'lamp',
                 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 
                 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 
                 'vase', 'wardrobe', 'xbox']
